# Run pipeline and create catalog of sources

In [ ]:
import os
os.environ["CRDS_PATH"] = "/Users/cpacifici/crds_cache_july2023/" # set appropriate path
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

import glob
import shutil

import jwst
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline

from astropy.table import Table

import numpy as np

import json

from jdaviz import Imviz
import warnings

In [ ]:
print('jwst:', jwst.__version__)

In [ ]:
# Update appropriately
data_dir_in = "/Users/cpacifici/DATA/NGDEEP/rate/"
data_dir_asn_level2 = "/Users/cpacifici/DATA/NGDEEP/asn_level2/"
data_dir_asn_level3 = "/Users/cpacifici/DATA/NGDEEP/asn_level3/"
data_dir_out_image2 = "/Users/cpacifici/DATA/NGDEEP/image2/"
data_dir_out_image3 = "/Users/cpacifici/DATA/NGDEEP/image3/"

## Read list of rate files for imaging

In [ ]:
listrate_file = './list_ngdeep_rate.csv'
listrate = Table.read(listrate_file)
listrate[np.where(listrate['FILTER']=='CLEAR')]

## Run image2 on all CLEAR images using the provided association files

### Check an association file for level 2

In [ ]:
asnfile = data_dir_asn_level2 + 'jw02079-o004_20230622t175524_image2_00001_asn.json'
asn_data = json.load(open(asnfile))
print(asn_data['products'][0]['members'][0]['expname'])
asn_data

**Developer note:** I do not see a way to tell which association file looks at which rate file without opening it and I believe association files and rate files need to be in the same directory where the pipeline is running, so I am going to temporarily copy all association file and all rate files in here and delete them when image2 is done.

### Move all needed files to the current directory

In [ ]:
asn_image2 = glob.glob(data_dir_asn_level2+'*image2*')
for file in asn_image2:
    shutil.copy(file, './')

In [ ]:
rate_clear = listrate[np.where(listrate['FILTER']=='CLEAR')]['FILENAME']
for file in rate_clear:
    shutil.copy(data_dir_in+file, './')

### Run image2

In [ ]:
asn_image2_local = glob.glob('./jw02079-o004_*_image2_*_asn.json')
asn_image2_local

In [ ]:
for file in asn_image2_local:
    print(file)
    image2 = Image2Pipeline()
    result = image2.call(file,
                         save_results=True,
                         output_dir=data_dir_out_image2) # Use the defaults for now

### Delete association and rate files from local directory

In [ ]:
dir_to_be_del = './files_to_be_deleted'
if not os.path.exists(dir_to_be_del):
    os.mkdir(dir_to_be_del)
    
for file in asn_image2_local:
    shutil.move(file, dir_to_be_del)
    
for file in glob.glob('./*rate.fits'):
    shutil.move(file, dir_to_be_del)
    
shutil.rmtree(dir_to_be_del)

## Run image3 for the three filters and create catalogs

### Check an association file for level 3

In [ ]:
asnfile = data_dir_asn_level3 + 'jw02079-o004_20230622t175524_image3_00002_asn.json'
asn_data = json.load(open(asnfile))
asn_data

### Move all needed files to the current directory

In [ ]:
asn_image3 = glob.glob(data_dir_asn_level3+'*image3*')
for file in asn_image3:
    shutil.copy(file, './')

In [ ]:
cal_clear = glob.glob(data_dir_out_image2+'*')
for file in cal_clear:
    shutil.copy(file, './')

### Run image3

In [ ]:
asn_image3_local = glob.glob('./*image3*.json')
asn_image3_local

I'll first run the step with the default options, check the catalog and rerun with adjusted parameters.

In [ ]:
for file in asn_image3_local:
    print(file)
    image3 = Image3Pipeline()
    result = image3.call(file,
                         steps={
                                'source_catalog':{'kernel_fwhm':5.0,
                                                  'snr_threshold':10.0,
                                                  'npixels':50,
                                                  'deblend':True,
                                                 },
                                'tweakreg':{'snr_threshold':20,
                                            'abs_refcat':'GAIADR2',
                                            'save_catalogs':True,
                                            'searchrad':3.0,
                                            'kernel_fwhm':2.302,
                                            'fitgeometry':'shift',
                                           },
                               },
                         save_results=True,
                         output_dir=data_dir_out_image3)

### Inspect the catalogs used for tweakreg

In [ ]:
cal_files = glob.glob(data_dir_out_image2+'*')
cat_files = []
for file in cal_files:
    filename = file.split('/')[-1]
    catname = filename.replace('cal.fits', 'cal_cat.ecsv')
    cat_files.append(catname)

In [ ]:
calexposure = 2
imviz_cal = Imviz()
viewer = imviz_cal.default_viewer
imviz_cal.load_data(cal_files[calexposure])
plotopt = imviz_cal.plugins['Plot Options']
plotopt.stretch_preset = '99%'
imviz_cal.show()

In [ ]:
catalog = Table.read(cat_files[calexposure])
t_xy = Table({'x': catalog['x'],
              'y': catalog['y']})

viewer.marker = {'color': 'orange', 'alpha': 1, 'markersize': 20, 'fill': False}
viewer.add_markers(t_xy)

### Delete not-needed files from local directory

List includes: cal, outlier_i2d, cal_cat, and asn.

In [ ]:
dir_to_be_del = './files_to_be_deleted'
if not os.path.exists(dir_to_be_del):
    os.mkdir(dir_to_be_del)
    
list_files = glob.glob('./jw02079*')

print("CHECK that list is ok before continuing")
list_files

In [ ]:
for file in list_files:
    shutil.move(file, dir_to_be_del)

shutil.rmtree(dir_to_be_del)

## Visualize catalog and i2d images in Imviz

In [ ]:
image3_i2d = [data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f115w_i2d.fits',
              data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f150w_i2d.fits',
              data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f200w_i2d.fits']

image3_segm = [data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f115w_segm.fits',
               data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f150w_segm.fits',
               data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f200w_segm.fits']

image3_cat = [data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f115w_cat.ecsv',
              data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f150w_cat.ecsv',
              data_dir_out_image3 + 'jw02079-o004_t001_niriss_clear-f200w_cat.ecsv']

image3_label = ['F115W',
                'F150W',
                'F200W']

In [ ]:
imviz = Imviz()
for ii in range(len(image3_i2d)):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(image3_i2d[ii], data_label=image3_label[ii])
imviz.show()

In [ ]:
viewer = imviz.default_viewer

In [ ]:
linking = imviz.plugins['Links Control']
linking.link_type = 'WCS'

In [ ]:
plotopt = imviz.plugins['Plot Options']
plotopt.multiselect = True
plotopt.select_all(viewers=True, layers=True)
plotopt.stretch_preset = '99.5%'
plotopt.multiselect = False